# EXTRACT 

In [1]:
import awswrangler as awr
import logging
import pandas as pd
import os

logging.basicConfig(
    level=logging.INFO,  # Exibe mensagens a partir de INFO
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler()  # Garante logs no console
    ]
)

logging.info('\n ----------------------------------------------------------------------------------')
logging.info('\n Executando Rotina: Movimentação de Placas')

class Extract:

    def __init__(self):
        self.path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes"

    def extract_all_ativacoes(self):

        try:
            dir_query = os.path.join(self.path, 'sql', 'all_boards_ATIVOS.sql')
            with open(dir_query, 'r') as file:
                query = file.read()
            df_ativacoes = awr.athena.read_sql_query(query, database='silver')
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de ativações extraída com sucesso!')
            return df_ativacoes
        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair a consulta de ativações: {e}')
            return None

    def extract_all_cancelamentos(self):

        try:
            dir_query = os.path.join(self.path, 'sql', 'all_boards_CANCELADOS.sql')
            with open(dir_query, 'r') as file:
                query = file.read()
            df_cancelamentos = awr.athena.read_sql_query(query, database='silver')
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de cancelamentos extraída com sucesso!')
            return df_cancelamentos
        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair a consulta de cancelamentos: {e}')
            return None

extract = Extract()
df_ativacoes = extract.extract_all_ativacoes()
df_cancelamentos = extract.extract_all_cancelamentos()



2025-08-26 11:23:10,188 - INFO - 
 ----------------------------------------------------------------------------------
2025-08-26 11:23:10,188 - INFO - 
 Executando Rotina: Movimentação de Placas
2025-08-26 11:23:10,221 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:12,225 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:13,710 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:16,171 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:18,358 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:20,347 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:22,240 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:25,193 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:27,1

# TRANSFORM

In [4]:
# IMPORTANDO MÓDULOS E PACOTES
import pandas as pd
import numpy as np
import datetime as dt
import logging


class Transform:
        
    def __init__(self) -> None:
        pass

    def transforming_files(self):

#DEFININDO DATAFRAMES VAZIOS 
        try:

            df_final_ativacoes = pd.DataFrame()
            df_final_cancelamentos = pd.DataFrame()

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('Falha ao definir dataframes.')

#TRANSFORMANDO DF_ATIV E SEGMENTANDO POR EMPRESA
        try:
            extract = Extract()
            df_final_cancelamentos = extract.extract_all_cancelamentos()
            df_ativ_all_boards = extract.extract_all_ativacoes()

            df_ativ_all_boards['data_ativacao_beneficio'] = pd.to_datetime(df_ativ_all_boards['data_ativacao_beneficio']).dt.date
                        
            df_ativ_all_boards['beneficio'] = df_ativ_all_boards['beneficio'].replace('REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO', 'CASCO (VEÍCULO)').replace('REPARAÇÃO OU REPOSIÇÃO DO (SEMI)REBOQUE', 'CASCO (R/SR)').replace('REPARAÇÃO OU REPOSIÇÃO DO COMPLEMENTO', 'CASCO (COMPLEMENTO)')
            
            df_ativ_viavante = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Viavante']
            df_ativ_stcoop = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Stcoop']
            df_ativ_segtruck = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Segtruck']
            df_ativ_tag = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Tag']

            df_final_cancelamentos = df_final_cancelamentos

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')  
            logging.info(f'Falha ao realizar a segmentação dos dataframes: {e}')

# SELECIONANDO APENAS AS ATIVAÇÕES CORRESPONDENTES AOS BENEFICIOS 'CASCO' / 'TERCEIRO' POR UM REGEX PADRÃO
        try:
            ids_beneficios_segtruck = [2, 3, 4, 7, 24, 25, 26, 29]
            ids_beneficios_stcoop = [24, 25, 26, 29]
            ids_beneficios_viavante = [40, 41, 42, 45]
            ids_beneficios_tag = [2, 3, 4, 7, 24, 25, 26, 29, 34, 35, 36, 37, 38, 39]

            df_ativ_viavante = df_ativ_viavante.loc[df_ativ_viavante['id_beneficio'].isin(ids_beneficios_viavante)]
            df_ativ_stcoop = df_ativ_stcoop[df_ativ_stcoop['id_beneficio'].isin(ids_beneficios_stcoop)]
            df_ativ_segtruck = df_ativ_segtruck.loc[df_ativ_segtruck['id_beneficio'].isin(ids_beneficios_segtruck)]
            df_ativ_tag = df_ativ_tag.loc[df_ativ_tag['id_beneficio'].isin(ids_beneficios_tag)]

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')  
            logging.info(f'Falha ao padronizar nomenclaturas referente aos beneficios pré-estabelecidos: {e}')

# CONCATENANDO E CRIANDO COLUNA DE MIGRAÇÃO (MIGRATION_FROM) 
        try:

            df_final_ativacoes = pd.concat([df_ativ_viavante, df_ativ_stcoop, df_ativ_segtruck, df_ativ_tag])

            if not df_final_ativacoes.empty:
                df_final_ativacoes['migration_from'] = np.nan

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha na criação da coluna de migração e concatenação de dataframes: {e}')

    # DEFININDO COLUNAS QUE SERÃO UTILIZADAS NO DATAFRAME FINAL
        try:
           
            df_final_ativacoes = df_final_ativacoes[[
                'placa', 'chassi', 'id_placa', 'id_veiculo', 'id_carroceria', 'matricula', 'conjunto', 'unidade', 'consultor', 'status_beneficio', 
                'cliente', 'data_registro', 'data_ativacao_beneficio', 'suporte', 'data_filtro', 'empresa', 'migration_from'
            ]]

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Processo de seleção de colunas realizado com sucesso!')

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao definir colunas: {e}')

    # RETIRANDO DUPLICATAS
        try:
            df_final_ativacoes = df_final_ativacoes.drop_duplicates(subset=['chassi'])

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Duplicatas retiradas com sucesso.')

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao retirar duplicatas. Revise o código: {e}')

# TRATANDO DADOS NULOS NOS DATAFRAMES
        try:
            df_final_ativacoes['placa'] = df_final_ativacoes['placa'].fillna('SEM-PLACA')
            df_final_ativacoes['chassi'] = df_final_ativacoes['chassi'].fillna('NULL')
            df_final_ativacoes['id_placa'] = df_final_ativacoes['id_placa'].fillna(0)
            df_final_ativacoes['id_veiculo'] = df_final_ativacoes['id_veiculo'].fillna(0)
            df_final_ativacoes['id_carroceria'] = df_final_ativacoes['id_carroceria'].fillna(0)
            df_final_ativacoes['matricula'] = df_final_ativacoes['matricula'].fillna(0)
            df_final_ativacoes['conjunto'] = df_final_ativacoes['conjunto'].fillna(0)
            df_final_ativacoes['unidade'] = df_final_ativacoes['unidade'].fillna('NULL')
            df_final_ativacoes['consultor'] = df_final_ativacoes['consultor'].fillna('NULL')
            df_final_ativacoes['status_beneficio'] = df_final_ativacoes['status_beneficio'].fillna('NULL')
            df_final_ativacoes['cliente'] = df_final_ativacoes['cliente'].fillna('NULL')
            df_final_ativacoes['data_registro'] = df_final_ativacoes['data_registro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['data_ativacao_beneficio'] = df_final_ativacoes['data_ativacao_beneficio'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['suporte'] = df_final_ativacoes['suporte'].fillna('NULL')
            df_final_ativacoes['data_filtro'] = df_final_ativacoes['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['empresa'] = df_final_ativacoes['empresa'].fillna('NULL')
            df_final_ativacoes['migration_from'] = df_final_ativacoes['migration_from'].fillna('NULL')

            df_final_cancelamentos['placa'] = df_final_cancelamentos['placa'].fillna('SEM-PLACA')
            df_final_cancelamentos['chassi'] = df_final_cancelamentos['chassi'].fillna('NULL')
            df_final_cancelamentos['id_placa'] = df_final_cancelamentos['id_placa'].fillna(0)
            df_final_cancelamentos['id_veiculo'] = df_final_cancelamentos['id_veiculo'].fillna(0)
            df_final_cancelamentos['id_carroceria'] = df_final_cancelamentos['id_carroceria'].fillna(0)
            df_final_cancelamentos['matricula'] = df_final_cancelamentos['matricula'].fillna(0)
            df_final_cancelamentos['conjunto'] = df_final_cancelamentos['conjunto'].fillna(0)
            df_final_cancelamentos['unidade'] = df_final_cancelamentos['unidade'].fillna('NULL')
            df_final_cancelamentos['status'] = df_final_cancelamentos['status'].fillna('NULL')
            df_final_cancelamentos['cliente'] = df_final_cancelamentos['cliente'].fillna('NULL')
            df_final_cancelamentos['data'] = df_final_cancelamentos['data'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos['data_cancelamento'] = df_final_cancelamentos['data_cancelamento'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos['usuario_cancelamento'] = df_final_cancelamentos['usuario_cancelamento'].fillna('NULL')
            df_final_cancelamentos['data_filtro'] = df_final_cancelamentos['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos['empresa'] = df_final_cancelamentos['empresa'].fillna('NULL')
            df_final_cancelamentos['migracao'] = df_final_cancelamentos['migracao'].fillna('NULL')
            df_final_cancelamentos['renegociacao'] = df_final_cancelamentos['renegociacao'].fillna('NULL')
            df_final_cancelamentos['data_substituicao'] = df_final_cancelamentos['data_substituicao'].fillna(pd.Timestamp('1900-01-01'))

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Processo de Transformacao de Dados concluido com sucesso!')

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao realizar tratamento de dados: {e}')
     
        return df_final_ativacoes, df_final_cancelamentos
    
transform = Transform()
df_final_ativacoes, df_final_cancelamentos = transform.transforming_files()

        

2025-08-26 15:36:58,810 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 15:36:59,754 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 15:37:01,179 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 15:37:02,702 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 15:37:03,881 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 15:37:06,023 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 15:37:07,992 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 15:37:10,378 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 15:37:12,871 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 15:37:15,161 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 15:37:17,

In [5]:
df_final_ativacoes_ativos = df_final_ativacoes[df_final_ativacoes['status_beneficio']=='ATIVO']
df_final_ativacoes_ativos.shape[0]

32913

In [6]:
df_final_ativacoes_ativos.head()

,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,consultor,status_beneficio,cliente,data_registro,data_ativacao_beneficio,suporte,data_filtro,empresa,migration_from
31502,NCX7F77,9BSP6X200B3688932,8,8,0,1,10097,UNIDADE CASCAVEL,Cesar Augusto Magalhaes Silva,ATIVO,CL ARAUJO TRANSPORTES LTDA,2025-02-13,2025-03-12,Lucas Lobo Didur,2025-08-26,Viavante,NULL
31505,AVC0F73,9BVAS50DXBE785089,10,10,0,3,10148,UNIDADE CASCAVEL,Aline Dayla Polove Zagonel,ATIVO,DARCI DALBOSCO,2025-02-14,2025-03-15,VICTOR GABRIEL GOMES PICÃO,2025-08-26,Viavante,NULL
31506,ASU6A71,9BVAS02C7AE761208,13,13,0,4,20919,UNIDADE CASCAVEL,Thiago Rosa De Figueiredo,ATIVO,LUAN DIAS GRECCO,2025-07-21,2025-07-21,Diego Alejandro Cortes Moya,2025-08-26,Viavante,NULL
31507,AUN1I97,9AA07102GBC105405,10,0,10,5,10158,UNIDADE CASCAVEL,Cesar Augusto Magalhaes Silva,ATIVO,ALP DE SOUZA TRANSPORTES LTDA,2025-02-14,2025-03-20,Camilla Minho,2025-08-26,Viavante,NULL
31509,ARU3D44,9BVAS02C3BE767153,14,14,0,5,10158,UNIDADE CASCAVEL,Cesar Augusto Magalhaes Silva,ATIVO,ALP DE SOUZA TRANSPORTES LTDA,2025-02-14,2025-03-20,Camilla Minho,2025-08-26,Viavante,NULL


# LOAD

In [ ]:
# IMPORTANDO MÓDULOS E PACOTES
import logging
import openpyxl
import os
import pandas as pd
import logging


file_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_acompanhamento\template\placas_movimentacoes.xlsx"
destination_dir = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas"
destination_path = os.path.join(destination_dir, os.path.basename(file_path))

with pd.ExcelWriter(destination_path, engine='openpyxl') as writer:
    df_final_ativacoes.to_excel(writer, index=False, sheet_name='ATIVAÇÕES')


logging.info('\n ----------------------------------------------------------------------------------')
logging.info('\n Processo de Carregamento de Dados concluido com sucesso!')




    


2025-08-26 09:55:58,998 - INFO - 
 ----------------------------------------------------------------------------------
2025-08-26 09:55:59,000 - INFO - 
 Processo de Carregamento de Dados concluido com sucesso!
